## Use OpenMM to Verify Your MM Functional Form Implementation

Recently, there have been efforts where MM functional forms are implemented within machine learning packages using tensor-accelerating framework to assist the differentiable parametrization of force fields, such as [Espaloma](https://github.com/choderalab/espaloma) and [JAX-MD](https://github.com/jax-md/jax-md).
When you implement your own MM functional forms, you might want to make sure that they match what are used in, say, OpenMM, exactly.
Since the nonbonded part of the energy requires a lot more love to implement exactly, in this tutorial we first show you how to check your bonded energy implementation.

For simplicity, we only work with small molecules for now.

In [1]:
import openmm
from openff.toolkit.topology import Molecule

ModuleNotFoundError: No module named 'openff'

ModuleNotFoundError: No module named 'openff'